In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

In [2]:
# Add warehouse data
entry2_data = pd.read_csv("../entry_loc2.csv")
entry2_data = entry2_data.dropna()
entry2_data['HD entry'] = 1*(entry2_data['HD'] > 0)
entry2_data['LOW entry'] = 1*(entry2_data['LOW'] > 0)
entry2_data['log income'] = np.log(1 + entry2_data['income_per_capita'])
entry2_data['log population'] = np.log(1 + entry2_data['population'])
entry2_data['log hd warehouse'] = np.log(1 + entry2_data['hd warehouse distance'])
entry2_data['log low warehouse'] = np.log(1 + entry2_data['low warehouse distance'])
entry2_data = entry2_data[(entry2_data['population']>=10000) & (entry2_data['income_per_capita']>=15000)]
entry2_data = entry2_data[(entry2_data['lon'] <= -30) & (entry2_data['lat'] >= 25) ]
entry2_data = sm.add_constant(entry2_data)

/home/erichschulman/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [4]:
class NashLogit(GenericLikelihoodModel):
    
    def nloglikeobs(self, params):
        n = self.exog.shape[0]
        k = int(self.exog.shape[1]/2)
        p = np.zeros((2, 2, n))

        for a_j in [0, 1]:
            util1 = np.dot(self.exog[:, 0:k], params[0:k]) + params[-3]*a_j
            util2 = np.dot(self.exog[:, k:2*k],params[k:2*k]) + params[-2]*a_j
            p[0, a_j, :] = 1 / (1 + np.exp(util1))
            p[1, a_j, :] = 1 / (1 + np.exp(util2))
        
        #choose lambda
        lamb = .5
        delta_pos = 1*(params[-3] >= 0)*(params[-2] >= 0)
        delta_neg = 1*(params[-3] <= 0)*(params[-2] <= 0)

        if (delta_pos == 0) and (delta_neg == 0):
            params[-2] = 0
            
        #constrain delta2 to be pos...
        
        #rho = np.clip(params[-1],-1,1)
        rho = 0
        
        # solve for probablity of nash
        prob01 = (p[0, 1, :])*(1 - p[1, 0, :])
        prob10 = (1 - p[0, 0, :])*(p[1, 1, :])
        prob00 = p[0, 0, :] * p[1, 0, :] 
        prob11 = (1 - p[0, 1, :])*(1 - p[1, 1, :])
        mult_eq =  (p[0, 1, :] - p[0, 0, :])* (p[1, 1, :] - p[1, 0, :])
        #introduce the copula
        mult_eq_rho = mult_eq*(1+rho*mult_eq)
        prob01_rho = prob01*(1+rho*prob10) - delta_neg*.5*mult_eq
        prob10_rho = prob10*(1+rho*prob01) - delta_neg*.5*mult_eq
        prob00_rho = prob00*(1+rho*prob11) - delta_pos*.5*mult_eq
        prob11_rho = prob11*(1+rho*prob00) - delta_pos*.5*mult_eq
               
        # compute empirical likelihood
        p00 = (1 - self.endog[:, 0])*(1 - self.endog[:, 1])
        p11 = self.endog[:, 0]*self.endog[:, 1]
        p10 = self.endog[:, 0] * (1 - self.endog[:, 1])
        p01 = (1 - self.endog[:, 0]) * self.endog[:, 1]
        

        ll = p00 * prob00_rho + p11 * prob11_rho + p01 * prob01_rho + p10 * prob10_rho
        
        return -np.log(np.maximum(ll, 1e-12))
        

    def fit(self, **kwds):
        """fit the likelihood function using the right start parameters"""
        k = int(self.exog.shape[1]/2)
        x1 = np.concatenate( (self.exog[:, 0:k], self.endog[:,1].reshape(self.endog.shape[0],1) ) ,axis=1)
        x2 = np.concatenate( (self.exog[:, k:2*k], self.endog[:,0].reshape(self.endog.shape[0],1) ),axis=1)
        params1 =  sm.Logit(self.endog[:, 0], x1).fit().params
        params2 = sm.Logit(self.endog[:, 1], x2).fit().params
        start_params = np.concatenate((params1[0:-1],params2[0:-1],[params1[-1],params2[-1], 0] ))
        #start_params = np.concatenate((params1[0:-1],params2[0:-1],[-1,-1,.8] ))

        print(start_params)
        return super(NashLogit, self).fit(start_params=start_params, **kwds)

        
        
x = entry2_data[['const','log income','log population','log hd warehouse',
                 'const','log income','log population','log low warehouse']].copy()
y = entry2_data[['HD entry','LOW entry']]

NashLogit_model = NashLogit(y,x).fit()
NashLogit_model.summary()

Optimization terminated successfully.
         Current function value: 0.516223
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.521237
         Iterations 6
[-1.09364059e+01 -5.08298059e-03  9.45608270e-01  8.33899479e-03
  4.47877722e+00 -1.15220618e+00  5.79471773e-01  5.06419108e-03
  1.33141327e+00  1.35747936e+00  0.00000000e+00]


/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                  NashLogit Results                                  
=====================================================================================
Dep. Variable:     ['HD entry', 'LOW entry']   Log-Likelihood:                -3166.0
Model:                             NashLogit   AIC:                             6342.
Method:                   Maximum Likelihood   BIC:                             6372.
Date:                       Fri, 16 Jul 2021                                         
Time:                               17:00:12                                         
No. Observations:                       2952                                         
Df Residuals:                           2947                                         
Df Model:                                  4                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               -12.0249        nan        nan        nan         nan         nan
log income           -0.0140        nan        nan        nan         nan         nan
log population        1.0959        nan        nan        nan         nan         nan
log hd warehouse      0.0107        nan        nan        nan         nan         nan
const                 4.5501        nan        nan        nan         nan         nan
log income           -1.1865        nan        nan        nan         nan         nan
log population        0.6146        nan        nan        nan         nan         nan
log low warehouse     0.0016        nan        nan        nan         nan         nan
par0                  0.2343        nan        nan        nan         nan         nan
par1                  1.1919        nan        nan        nan         nan         nan
par2                 -0.0035        nan        nan        nan         nan         nan
=====================================================================================
"""